In [2]:
import bezier
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import networkx as nx
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import itertools
import functools
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
%load_ext autoreload
%autoreload 2

In [3]:
class GraphGram(object):
    
    def __init__(self, graph, xoff=0, yoff=0, scale=1, layout_method='kamada_kawai_layout'):
        self.graph = graph
        self._nodes = None
        self.xoff = xoff
        self.yoff = yoff
        self.scale = scale
        self.layout_method = layout_method
    
    @property
    def center(self):
        return np.array((self.xoff, self.yoff))
    
    @property
    def edges(self):
        return list(self.graph.edges)
    
    @property
    def layout_function(self):
        try:
            f = getattr(nx.layout, self.layout_method)
        except AttributeError:
            layout_functions = [a for a in dir(nx.layout) if 'layout' in a]
            error_string = f'''{self.layout_method} not found in networkx.layout module; 

choose from {layout_functions}
            '''
            print(error_string)
        return f
    
    @functools.lru_cache
    def get_layout(self, *args, **kwargs):
        self._nodes = self.layout_function(
            self.graph,
            scale=self.scale,
            center=self.center,
            *args, **kwargs)
    
    @property
    def nodes(self):
        if self._nodes is None:
            self.get_layout()
        return self._nodes
    
    @property
    def node_pts(self):
        return {k:Point(xy) for k, xy in self.nodes.items()}
    
    @property
    def pts(self):
        return MultiPoint([p for p in self.node_pts.values()])
    
    @property
    def lines(self):
        lines = []
        for n0,n1 in self.edges:
            p0 = self.node_pts[n0]
            p1 = self.node_pts[n1]
            lines.append(LineString([p0, p1]))
        return MultiLineString(lines)    
        

# strogatz

In [235]:
# make page
paper_size = '11x17 inches'
border:float=20
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [749]:
DEGREE = 13
SCALE = 15
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=40, ystep=40, flatmesh=True)

In [750]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0.2, 1.] )
_p_gen = gp.make_callable(p_gen)

In [751]:
k_gen = 3
_k_gen = gp.make_callable(k_gen)

In [752]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [753]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='kamada_kawai_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    new_row = row.to_dict()
    new_row['geometry'] = gg.lines
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [755]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

tolerance=0.5
sk.vpype(f'linesort')

sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2020-12-28T10:28:58.963892

In [756]:
savedir='/mnt/c/code/side/plotter_images/oned_outputs'
filename = '0149_graph_test.svg'

In [757]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# strogatz

In [977]:
# make page
paper_size = '11x17 inches'
border:float=18
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [984]:
DEGREE = 33
SCALE = 8
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=20, ystep=20, flatmesh=True)

In [985]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0., 0.6] )
_p_gen = gp.make_callable(p_gen)

In [986]:
k_gen = 2
_k_gen = gp.make_callable(k_gen)

In [987]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [988]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spring_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=2)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [989]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

tolerance= 0.5
sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')

sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2020-12-28T16:48:31.746169 
 
 
 
 
 
 <polygon points="92.2958,112.9488 95.126,107.655 95.126,107.655 97.345,102.1476 97.345,102.1476 98.6455,96.5005 98.6455,96.5005 98.6567,90.8882 98.6567,90.8882 97.099,85.5968 97.099,85.5968 94.08,81.0603 94.08,81.0603 89.9181,77.4587 89.9181,77.4587 84.9999,74.8453 84.9999,74.8453 79.5874,73.2903 79.5874,73.2903 73.9896,72.4921 73.9896,72.4921 68.3672,72.4584 68.3672,72.4584 62.8916,73.2232 62.8916,73.2232 57.7755,74.8244 57.7755,74.8244 53.3295,77.343 53.3295,77.343 50.0864,80.8926 50.0864,80.8926 48.8507,85.3293 48.8507,85.3293 50.0165,89.8349 50.0165,89.8349 53.085,93.715 53.085,93.715 56.9192,96.7845 56.9192,96.7845 60.7486,99.4712 60.7486,99.4712 64.6645,102.0021 64.6645,102.0021 68.5873,104.5736 68.5873,104.5736 71.5889,107.8987 71.5889,107.8987 72.4114,112.2075 72.4114,112.2075 71.4066,116.6953 71.4066,116.6953 69.9015,121.118 69.9015,121.118 69.662,125.0436 69.662,125.0436 72.2192,127.5298 72.2192,127.5298 76.9198,128.126 76.9198,128.126 81.4188,126.0504 81.4188,126.0504 85.4301,122.4206 85.4301,122.4206 89.0476,117.9365 89.0476,117.9365"/>
 <polyline points="130.9681,92.0672 128.272,97.1205 128.272,97.1205 127.54,102.4449 127.54,102.4449 129.4092,106.9801 129.4092,106.9801 133.4766,108.8931 133.4766,108.8931 136.2517,106.1931 136.2517,106.1931 135.0157,101.5944 135.0157,101.5944 132.8218,96.7644 132.8218,96.7644 130.9681,92.0672 130.9681,92.0672 132.8652,85.409 132.8652,85.409 135.879,79.4741 135.879,79.4741 139.3651,74.2564 139.3651,74.2564 143.2685,70.0732 143.2685,70.0732 147.64,67.6535 147.64,67.6535 152.0037,68.515 152.0037,68.515 154.5478,72.5224 154.5478,72.5224 155.1896,77.9546 155.1896,77.9546 154.5744,84.0 154.5744,84.0 153.3462,90.4244 153.3462,90.4244 156.5372,96.0644 156.5372,96.0644 160.521,101.1782 160.521,101.1782 165.0849,105.8409 165.0849,105.8409 165.1387,109.7414"/>
 
 
 <polyline points="138.6111,182.2147 139.785,178.519 139.785,178.519 140.9893,173.6149 140.9893,173.6149 141.9435,167.9575 141.9435,167.9575 143.263,162.1479 143.263,162.1479 145.4993,157.2493 145.4993,157.2493 148.9289,153.9362 148.9289,153.9362 153.1673,153.3885 153.1673,153.3885 156.3882,156.2442 156.3882,156.2442 157.5932,160.9216 157.5932,160.9216 163.9399,161.5758 163.9399,161.5758 169.4725,163.2561 169.4725,163.2561 173.9241,166.3128 173.9241,166.3128 176.6979,170.6944 176.6979,170.6944 177.4743,175.8327 177.4743,175.8327 176.4702,181.0708 176.4702,181.0708 174.0522,185.957 174.0522,185.957 170.6019,190.2403 170.6019,190.2403 166.4374,193.7709 166.4374,193.7709 161.8611,196.491 161.8611,196.491 157.1874,198.4392 157.1874,198.4392 152.8541,199.8876 152.8541,199.8876 149.4899,201.2509"/>
 
 <polygon points="218.665,155.908 221.9125,160.3254 221.9125,160.3254 225.3002,165.3332 225.3002,165.3332 229.1649,170.3141 229.1649,170.3141 233.8456,174.4422 233.8456,174.4422 239.4166,176.9394 239.4166,176.9394 240.6675,171.8437 240.6675,171.8437 241.1206,166.6135 241.1206,166.6135 241.4684,161.7278 241.4684,161.7278 246.6208,163.5704 246.6208,163.5704 249.2719,168.1756 249.2719,168.1756 248.7413,173.9777 248.7413,173.9777 245.8742,179.9431 245.8742,179.9431 241.1735,185.6835 241.1735,185.6835 236.02,190.5112 236.02,190.5112 230.2264,193.9788 230.2264,193.9788 223.9814,195.8245 223.9814,195.8245 217.6293,195.8365 217.6293,195.8365 211.6263,193.9912 211.6263,193.9912 206.4475,190.4008 206.4475,190.4008 202.4637,185.4221 202.4637,185.4221 199.6547,179.5454 199.6547,179.5454 200.4737,172.9838 200.4737,172.9838 202.2056,166.748 202.2056,166.748 204.6126,161.05 204.6126,161.05 206.8301,156.0796 206.8301,156.0796 210.2179,152.7721 210.2179,152.7721 214.8528,152.8601 214.8528,152.8601"/>
 
 
 
 <polygon points="296.0373,167.4978 290.7457,170.5751 290.7457,170.5751 285.3617,171.6993 285.3617,171.6993 280.1494,171.0794 280.1494,171.0794 275.51,169.656 275.51,169.656 273.6603,166.9725 273.6603,166.9725 277.1701,168.4032 277.1701,168.4032 282.398

In [990]:
savedir='/mnt/c/code/side/plotter_images/oned_outputs'
filename = '0151_strogatz_graphs.svg'

In [991]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# binary tree

In [3]:
# make page
paper_size = '11x17 inches'
border:float=18
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [48]:
DEGREE = 33
SCALE = 20
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=50, ystep=50, flatmesh=True)

In [49]:
r_gen = lambda x: int(np.interp(x, [xs.min(), xs.max()], [2, 8] ))
_r_gen = gp.make_callable(r_gen)

In [50]:
k_gen = 3
_k_gen = gp.make_callable(k_gen)

In [51]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [58]:
nx.number_of_nonisomorphic_trees(10)

106

In [52]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spectral_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=2)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [53]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

tolerance= 0.5
sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')

sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-01-10T22:01:38.867507 
 
 
 
 
 
 <polygon points="151.5235,78.9921 165.7978,80.3581 165.7978,80.3581 179.4996,84.4068 179.4996,84.4068 192.1338,90.9919 192.1338,90.9919 203.2436,99.8752 203.2436,99.8752 212.4276,110.7359 212.4276,110.7359 219.3539,123.1812 219.3539,123.1812 223.772,136.7615 223.772,136.7615 225.5223,150.9859 225.5223,150.9859 224.5416,165.3403 224.5416,165.3403 220.8653,179.3059 220.8653,179.3059 214.6262,192.378 214.6262,192.378 206.05,204.084 206.05,204.084 195.4464,214.0009 195.4464,214.0009 183.1988,221.7702 183.1988,221.7702 169.7499,227.1113 169.7499,227.1113 155.5856,229.8309 155.5856,229.8309 141.218,229.8309 141.218,229.8309 127.1663,227.1113 127.1663,227.1113 113.9385,221.7702 113.9385,221.7702 102.0125,214.0009 102.0125,214.0009 91.8194,204.084 91.8194,204.084 83.7276,192.378 83.7276,192.378 78.0296,179.3059 78.0296,179.3059 74.9314,165.3403 74.9314,165.3403 74.5448,150.9859 74.5448,150.9859 76.8839,136.7615 76.8839,136.7615 81.8641,123.1812 81.8641,123.1812 89.3055,110.7359 89.3055,110.7359 98.9391,99.8752 98.9391,99.8752 110.4166,90.9919 110.4166,90.9919 123.3233,84.4068 123.3233,84.4068 137.1927,80.3581 137.1927,80.3581"/>
 <polyline points="263.345,155.9939 265.4213,155.9441 265.4213,155.9441 269.5171,155.8462 269.5171,155.8462 275.5196,155.7038 275.5196,155.7038 283.2641,155.5218 283.2641,155.5218 292.5379,155.3067 292.5379,155.3067 303.0861,155.0661 303.0861,155.0661 314.6191,154.8083 314.6191,154.8083 326.82,154.5427 326.82,154.5427 339.3537,154.2784 339.3537,154.2784 344.1391,148.7723 344.1391,148.7723 348.7816,143.4713 348.7816,143.4713 353.4515,130.7811 353.4515,130.7811 357.7226,118.9308 357.7226,118.9308 361.4777,108.3385 361.4777,108.3385 364.6136,99.378 364.6136,99.378 367.0441,92.3653 367.0441,92.3653 368.7025,87.5481 368.7025,87.5481 369.5433,85.0962"/>
 
 
 
 
 
 <polyline points="518.9664,139.9135 519.1624,140.6653 519.1624,140.6653 519.55,142.1303 519.55,142.1303 531.6422,146.2361 531.6422,146.2361 543.6527,150.7696 543.6527,150.7696 555.3183,155.4986 555.3183,155.4986 560.5907,157.7105 560.5907,157.7105 565.1472,159.7621 565.1472,159.7621 568.8881,161.5483 568.8881,161.5483 571.7314,162.9775 571.7314,162.9775 570.4044,162.2706 570.4044,162.2706 572.3742,163.1751 573.37,163.6392 572.3742,163.1751 571.7314,162.9775 574.9416,164.6835 574.9416,164.6835 577.1217,165.8718 577.1217,165.8718 578.2238,166.4816"/>
 
 
 
 
 
 <polyline points="485.1977,354.5205 486.181,353.8326 486.181,353.8326 488.1248,352.4999 488.1248,352.4999 490.9845,350.606 490.9845,350.606 494.6943,348.2699 494.6943,348.2699 508.968,363.4207 508.968,363.4207 523.6777,377.3249 523.6777,377.3249 538.4849,389.1099 538.4849,389.1099 539.365,402.5172 539.365,402.5172 539.9814,412.2243 539.9814,412.2243 540.2658,416.8223 540.2658,416.8223 539.9814,412.2243 539.9814,412.2243 539.7513,408.4259 539.7513,408.4259 539.2486,400.5565 539.2486,400.5565 538.4849,389.1099 538.4849,389.1099 551.4051,373.1823 551.4051,373.1823 563.7844,355.3956 563.7844,355.3956 575.338,336.8659 575.338,336.8659 584.6825,319.2046 584.6825,319.2046 592.7201,303.0717"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <polyline points="863.9786,202.4699 862.6097,207.0399 862.6097,207.0399 863.9786,202.4699 863.9786,202.4699 868.0418,189.1582 868.0418,189.1582 873.302,172.8344 873.302,172.8344 879.5926,154.9205 879.5926,154.9205 891.1071,136.5849 891.1071,136.5849 905.5561,140.7169 905.5561,140.7169 920.0148,146.0568 920.0148,146.0568 924.7199,147.8812 924.7199,147.8812 928.8283,150.2894 928.8283,150.2894 932.21,153.0717 932.21,153.0717 934.758,155.9855 934.758,155.9855 936.3916,158.7772 937.3892,159.1775 936.3916,158.7772 936.3916,158.7772 936.0615,160.8031 936.0615,160.8031 934.7758,162.2872"/>
 
 
 
 <polyline points="894.0488,338.3023 885.356,350.5096 885.356,350.5096 883.2486,335.4238 883.2486,335.4238 881.759,320.8735 881.759,320.8735 883.1956,314.3492 883.1956,314.3492 888.3834,325.7393 88

In [990]:
savedir='/mnt/c/code/side/plotter_images/oned_outputs'
filename = '0151_strogatz_graphs.svg'

In [991]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# strogatz

In [4]:
# make page
paper_size = '23.42x16.92 inches'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [984]:
DEGREE = 33
SCALE = 8
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=20, ystep=20, flatmesh=True)

In [985]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0., 0.6] )
_p_gen = gp.make_callable(p_gen)

In [986]:
k_gen = 2
_k_gen = gp.make_callable(k_gen)

In [987]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [988]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spring_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=2)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [989]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

tolerance= 0.5
sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')

sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2020-12-28T16:48:31.746169 
 
 
 
 
 
 <polygon points="92.2958,112.9488 95.126,107.655 95.126,107.655 97.345,102.1476 97.345,102.1476 98.6455,96.5005 98.6455,96.5005 98.6567,90.8882 98.6567,90.8882 97.099,85.5968 97.099,85.5968 94.08,81.0603 94.08,81.0603 89.9181,77.4587 89.9181,77.4587 84.9999,74.8453 84.9999,74.8453 79.5874,73.2903 79.5874,73.2903 73.9896,72.4921 73.9896,72.4921 68.3672,72.4584 68.3672,72.4584 62.8916,73.2232 62.8916,73.2232 57.7755,74.8244 57.7755,74.8244 53.3295,77.343 53.3295,77.343 50.0864,80.8926 50.0864,80.8926 48.8507,85.3293 48.8507,85.3293 50.0165,89.8349 50.0165,89.8349 53.085,93.715 53.085,93.715 56.9192,96.7845 56.9192,96.7845 60.7486,99.4712 60.7486,99.4712 64.6645,102.0021 64.6645,102.0021 68.5873,104.5736 68.5873,104.5736 71.5889,107.8987 71.5889,107.8987 72.4114,112.2075 72.4114,112.2075 71.4066,116.6953 71.4066,116.6953 69.9015,121.118 69.9015,121.118 69.662,125.0436 69.662,125.0436 72.2192,127.5298 72.2192,127.5298 76.9198,128.126 76.9198,128.126 81.4188,126.0504 81.4188,126.0504 85.4301,122.4206 85.4301,122.4206 89.0476,117.9365 89.0476,117.9365"/>
 <polyline points="130.9681,92.0672 128.272,97.1205 128.272,97.1205 127.54,102.4449 127.54,102.4449 129.4092,106.9801 129.4092,106.9801 133.4766,108.8931 133.4766,108.8931 136.2517,106.1931 136.2517,106.1931 135.0157,101.5944 135.0157,101.5944 132.8218,96.7644 132.8218,96.7644 130.9681,92.0672 130.9681,92.0672 132.8652,85.409 132.8652,85.409 135.879,79.4741 135.879,79.4741 139.3651,74.2564 139.3651,74.2564 143.2685,70.0732 143.2685,70.0732 147.64,67.6535 147.64,67.6535 152.0037,68.515 152.0037,68.515 154.5478,72.5224 154.5478,72.5224 155.1896,77.9546 155.1896,77.9546 154.5744,84.0 154.5744,84.0 153.3462,90.4244 153.3462,90.4244 156.5372,96.0644 156.5372,96.0644 160.521,101.1782 160.521,101.1782 165.0849,105.8409 165.0849,105.8409 165.1387,109.7414"/>
 
 
 <polyline points="138.6111,182.2147 139.785,178.519 139.785,178.519 140.9893,173.6149 140.9893,173.6149 141.9435,167.9575 141.9435,167.9575 143.263,162.1479 143.263,162.1479 145.4993,157.2493 145.4993,157.2493 148.9289,153.9362 148.9289,153.9362 153.1673,153.3885 153.1673,153.3885 156.3882,156.2442 156.3882,156.2442 157.5932,160.9216 157.5932,160.9216 163.9399,161.5758 163.9399,161.5758 169.4725,163.2561 169.4725,163.2561 173.9241,166.3128 173.9241,166.3128 176.6979,170.6944 176.6979,170.6944 177.4743,175.8327 177.4743,175.8327 176.4702,181.0708 176.4702,181.0708 174.0522,185.957 174.0522,185.957 170.6019,190.2403 170.6019,190.2403 166.4374,193.7709 166.4374,193.7709 161.8611,196.491 161.8611,196.491 157.1874,198.4392 157.1874,198.4392 152.8541,199.8876 152.8541,199.8876 149.4899,201.2509"/>
 
 <polygon points="218.665,155.908 221.9125,160.3254 221.9125,160.3254 225.3002,165.3332 225.3002,165.3332 229.1649,170.3141 229.1649,170.3141 233.8456,174.4422 233.8456,174.4422 239.4166,176.9394 239.4166,176.9394 240.6675,171.8437 240.6675,171.8437 241.1206,166.6135 241.1206,166.6135 241.4684,161.7278 241.4684,161.7278 246.6208,163.5704 246.6208,163.5704 249.2719,168.1756 249.2719,168.1756 248.7413,173.9777 248.7413,173.9777 245.8742,179.9431 245.8742,179.9431 241.1735,185.6835 241.1735,185.6835 236.02,190.5112 236.02,190.5112 230.2264,193.9788 230.2264,193.9788 223.9814,195.8245 223.9814,195.8245 217.6293,195.8365 217.6293,195.8365 211.6263,193.9912 211.6263,193.9912 206.4475,190.4008 206.4475,190.4008 202.4637,185.4221 202.4637,185.4221 199.6547,179.5454 199.6547,179.5454 200.4737,172.9838 200.4737,172.9838 202.2056,166.748 202.2056,166.748 204.6126,161.05 204.6126,161.05 206.8301,156.0796 206.8301,156.0796 210.2179,152.7721 210.2179,152.7721 214.8528,152.8601 214.8528,152.8601"/>
 
 
 
 <polygon points="296.0373,167.4978 290.7457,170.5751 290.7457,170.5751 285.3617,171.6993 285.3617,171.6993 280.1494,171.0794 280.1494,171.0794 275.51,169.656 275.51,169.656 273.6603,166.9725 273.6603,166.9725 277.1701,168.4032 277.1701,168.4032 282.398

In [990]:
savedir='/mnt/c/code/side/plotter_images/oned_outputs'
filename = '0151_strogatz_graphs.svg'

In [991]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)